In [ ]:
import globo , os , glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import Adam
import keras.backend as K
from dataset import *
from utils import *
from model import *
from loss import RankingLoss
from train import *


In [ ]:
''' LOSS FUNCTIONS VERSUS '''

BATCH_SIZE = 4
lambda1 = lambda2 = 8e-5
nrm = tf.zeros([BATCH_SIZE, 32, 1])
#abn = tf.ones([BATCH_SIZE, 32, 1])
abn = tf.random.uniform([BATCH_SIZE, 32, 1] , maxval = 1)
scores = tf.concat((nrm, abn), axis=0)
print(scores.shape)


def smoothfx(arr , deepmil=False):
    if deepmil:
        arr2 = np.zeros_like(arr)
        arr2[:-1] = arr[1:]
        arr2[-1] = arr[-1]
    else: 
        arr2 = tf.concat([arr[1:], arr[-1:]], axis=0)
    
    loss = tf.reduce_sum(tf.square(arr2 - arr))
    return lambda1 * loss


def sparsityfx( arr):
    loss = tf.reduce_sum(arr)
    return lambda1 * loss


def loss_deepmil(scores):
    print("\n\nDEEPMIL")
    scores = tf.reshape(scores, (-1, 1))
    #print("reshape =",scores.shape)

    loss1 = tf.constant(0.0, dtype=tf.float32)
    loss2 = tf.constant(0.0, dtype=tf.float32)
    
    l1 = 0.0 ; l2 = 0.0
    for i in range(BATCH_SIZE):
        startn = i * 32
        endn = (i + 1) * 32
        #print(f'startn {startn} , endn {endn}')
        
        starta = (i * 32 + BATCH_SIZE * 32)
        enda = (i + 1) * 32 + BATCH_SIZE * 32
        #print(f'starta {starta} , enda {enda}')
        
        maxn = tf.reduce_max(scores[ startn : endn ])
        maxa = tf.reduce_max(scores[ starta : enda ])
        #print(maxn,maxn)
        
        tmp = tf.nn.relu(1.0 - maxa + maxn)
        loss1 += tmp
        loss1 += sparsityfx(scores[ starta : enda ])
        loss1 += smoothfx(scores[ starta : enda ] , False)
        
        loss2 += tmp
        loss2 += sparsityfx(scores[ starta : enda ])
        loss2 += smoothfx(scores[ starta : enda ] , True)
        
    l1 = (loss1 / BATCH_SIZE).numpy()
    l2 = (loss2 / BATCH_SIZE).numpy()
    print("l1",l1,"l2",l2)
    print("abs dif = ",abs(l1 - l2) , "\nisclose?" ,np.isclose(l1 , l2))
    return l1 , l2


def loss_milbert(scores):
    print("\nMILBERT")
    loss = tf.constant(0.0, dtype=tf.float32)
    sparsity = tf.constant(0.0, dtype=tf.float32)
    smooth = tf.constant(0.0, dtype=tf.float32)
    l=0.0
    for i in range(BATCH_SIZE):
        #print(i)
        normal_index = tf.random.shuffle(tf.range(32))
        y_normal  = tf.gather(scores[i], normal_index)
        y_normal_max = tf.reduce_max(y_normal)
        y_normal_min = tf.reduce_min(y_normal)
        #print("normal",normal_index ,'\n', y_normal ,'\n', y_normal_max.numpy() ,'\n', y_normal_min.numpy())
        
        #print(str(i + BATCH_SIZE))
        anomaly_index = tf.random.shuffle(tf.range(32))
        y_anomaly = tf.gather(scores[i + BATCH_SIZE], anomaly_index)
        y_anomaly_max = tf.reduce_max(y_anomaly)
        y_anomaly_min = tf.reduce_min(y_anomaly)
        #print("abnormal",anomaly_index ,'\n', y_anomaly ,'\n', y_anomaly_max.numpy() ,'\n', y_anomaly_min.numpy())
        
        ## original milbert
        ## sparsity uses anomaly scores shuffled
        ## smooth uses original anomaly scores
        loss += tf.nn.relu(1.0 - y_anomaly_max + y_normal_max) 
        sparsity += tf.reduce_sum(y_anomaly) * lambda1
        smooth += tf.reduce_sum(tf.square(scores[i + BATCH_SIZE, :31] - scores[i + BATCH_SIZE, 1:32])) * lambda1
        
    l = ((loss + sparsity + smooth ) / BATCH_SIZE).numpy()

    print("l",l)
    return l 
    
    
l1 , l11 = loss_deepmil(scores)
l2  = loss_milbert(scores)
#abs(l1 - l2)
print('\n', np.isclose(l1 , l11 , l2) )

#### RTFM FEATURES TRAIN

In [ ]:
p1 = '/raid/DATASETS/anomaly/UCF_Crimes/features/I3D_RTFM_10CROP/train_abnormal/Abuse001_x264_i3d.npy'
features = np.load(p1)
print(np.shape(features))

features = features.transpose(1, 0, 2)  # [10, B, T, F]
print(np.shape(features))
divided_features = []
for feature in features:
    feature = process_feat(feature, 32)  # divide a video into 32 segments
    print(np.shape(feature))
    divided_features.append(feature)
    
divided_features = np.array(divided_features, dtype=np.float32)
print(np.shape(divided_features))

#### GT RTFM

In [ ]:
p1 = 'list/gt-ucf_rtfm.npy'
features = np.load(p1)
print(np.shape(features))

#### GT

In [ ]:
a = np.zeros((3 , 10))
aa = np.ones_like(a)
aaa = np.append(a , aa)

b = np.zeros_like(aaa)

np.allclose(aaa[:3] , b[:3])

In [ ]:
aaa = np.load(globo.UCFCRIME_GT)
test_tframes = np.shape(aaa)[0]

ooo = open(globo.UCFCRIME_I3D_DEEPMIL_LISTS["test"])
alltframes = 0
for i , line in enumerate(ooo):
    fpath = line.strip('\n')
    feat = np.load(fpath)
    tsteps = np.shape(feat)[0]
    ftotal = tsteps * 16
    
    gtfp = aaa[alltframes : alltframes+ftotal]
    print(f'{fpath}\n{ftotal}\n{gtfp}')
    
    #if i in (139,140) : print( fpath , tsteps)
    alltframes += ftotal
    
print(alltframes)
assert alltframes ==  test_tframes

## somehow the i3d rtfm features are incomplete
#ooo = open(globo.UCFCRIME_I3D_RTFM_LISTS["test"])
#cnt = 0
#for line in ooo:
#    fn = line.strip('\n')
#    feat = np.load(fn)
#    print(feat)
#    ts = np.shape(feat)[0]
#    cnt += ts*16
#assert cnt ==  test_tframes
